# **CA3/CA1 Neuromorphic Module – Full Visual Demo (Colab‑Ready)**This notebook automatically clones your GitHub repository and runs fully in Google Colab.

In [ ]:
import osos.chdir("/content")!rm -rf CA3-CA1HippocampalModule!git clone https://github.com/Fideloub/CA3-CA1HippocampalModule.gitos.chdir("/content/CA3-CA1HippocampalModule")print("Repo ready.")

In [ ]:
import randomimport numpy as npimport matplotlib.pyplot as pltfrom backend.network import CA3CA1Networkfrom backend.simulation import Simulationfrom backend.plasticity import STDPfrom agents.input_agent import InputAgentfrom agents.monitoring_agent import MonitoringAgentfrom orchestrator.orchestrator import Orchestratornet = CA3CA1Network(n_ca3=100, n_ca1=100)net.connect()stdp = STDP()print("Network ready:", len(net.ca3), "CA3 neurons,", len(net.ca1), "CA1 neurons")

In [ ]:
def get_activity(neurons):    return np.array([1 if n.fired else 0 for n in neurons])def get_weights(synapses):    return np.array([s.weight for s in synapses])def present_pattern(pattern, net, stdp, steps=3):    orch = Orchestrator(net, [InputAgent(), MonitoringAgent()])    orch.run_cycle(pattern)    sim = Simulation(net)    sim.run(steps)    for syn in net.synapses:        stdp.apply(syn, syn.pre.fired, syn.post.fired)    return get_activity(net.ca3), get_activity(net.ca1)

In [ ]:
num_patterns = 5patterns = [[random.choice([0,1]) for _ in range(len(net.ca3))] for _ in range(num_patterns)]print("Generated patterns.")

In [ ]:
initial_weights = get_weights(net.synapses)plt.hist(initial_weights, bins=30)plt.title("Initial Synaptic Weights")plt.show()

In [ ]:
ca3_hist = []ca1_hist = []for i, p in enumerate(patterns):    print("Training pattern", i+1)    ca3, ca1 = present_pattern(p, net, stdp)    ca3_hist.append(ca3)    ca1_hist.append(ca1)ca3_hist = np.array(ca3_hist)ca1_hist = np.array(ca1_hist)

In [ ]:
plt.imshow(ca3_hist, aspect='auto')plt.title("CA3 Activity")plt.colorbar()plt.show()plt.imshow(ca1_hist, aspect='auto')plt.title("CA1 Activity")plt.colorbar()plt.show()

In [ ]:
final_weights = get_weights(net.synapses)plt.hist(final_weights, bins=30)plt.title("Final Synaptic Weights")plt.show()print("Initial mean:", float(np.mean(initial_weights)))print("Final mean:", float(np.mean(final_weights)))

In [ ]:
original = patterns[0]noisy = [v if random.random()>0.25 else 1-v for v in original]orch = Orchestrator(net, [InputAgent(), MonitoringAgent()])orch.run_cycle(noisy)sim = Simulation(net)sim.run(3)recalled = get_activity(net.ca1)plt.figure(figsize=(10,3))plt.subplot(1,3,1)plt.imshow([original], aspect='auto')plt.title("Original")plt.subplot(1,3,2)plt.imshow([noisy], aspect='auto')plt.title("Noisy")plt.subplot(1,3,3)plt.imshow([recalled], aspect='auto')plt.title("Recalled")plt.show()